In [1]:
#from spektral.datasets import citation
import spektral
import numpy as np
import scipy as sc
import matplotlib.pyplot as plt

from spektral.layers import GraphConv
from spektral import utils

from keras.models import Model
from keras.layers import Input, Dropout

Using TensorFlow backend.


### Source:
https://danielegrattarola.github.io/spektral/getting-started/

### Some test

In [2]:
data = spektral.datasets.citation.load_data('cora')
A, X, y_train, y_val, y_test, train_mask, val_mask, test_mask = data

N = A.shape[0]
F = X.shape[-1]
n_classes = y_train.shape[-1]

Loading cora dataset
Pre-processing node features


In [3]:
A.shape, X.shape, y_val.shape, y_test.shape, train_mask.shape, val_mask.shape, test_mask.shape

((2708, 2708), (2708, 1433), (2708, 7), (2708, 7), (2708,), (2708,), (2708,))

In [4]:
# Model definition
X_in = Input(shape=(F, ))  # Input layer for X
A_in = Input((N, ), sparse=True)  # Input layer for A

graph_conv_1 = GraphConv(16, activation='relu')([X_in, A_in])
dropout = Dropout(0.5)(graph_conv_1)
graph_conv_2 = GraphConv(n_classes, activation='softmax')([dropout, A_in])

# Build model
model = Model(inputs=[X_in, A_in], outputs=graph_conv_2)

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


In [5]:
A = utils.localpooling_filter(A)

In [6]:
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              weighted_metrics=['acc'])
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 1433)         0                                            
__________________________________________________________________________________________________
input_2 (InputLayer)            (None, 2708)         0                                            
__________________________________________________________________________________________________
graph_conv_1 (GraphConv)        (None, 16)           22944       input_1[0][0]                    
                                                                 input_2[0][0]                    
__________________________________________________________________________________________________
dropout_1 (Dropout)             (None, 16)           0           graph_conv_1[0][0]               
__________

In [8]:
# Train model
validation_data = ([X, A], y_val, val_mask)
model.fit([X, A],
          y_train,
          sample_weight=train_mask,
          epochs=100,
          batch_size=N,
          validation_data=validation_data,
          shuffle=False)  # Shuffling data means shuffling the whole graph

Instructions for updating:
Use tf.cast instead.
Train on 2708 samples, validate on 2708 samples
Epoch 1/100
2708/2708 [==============================] - 1s 537us/step - loss: 1.9461 - weighted_acc: 0.1429 - val_loss: 1.9452 - val_weighted_acc: 0.1820
Epoch 2/100
2708/2708 [==============================] - 0s 11us/step - loss: 1.9447 - weighted_acc: 0.2500 - val_loss: 1.9449 - val_weighted_acc: 0.2260
Epoch 3/100
2708/2708 [==============================] - 0s 9us/step - loss: 1.9447 - weighted_acc: 0.2071 - val_loss: 1.9446 - val_weighted_acc: 0.2700
Epoch 4/100
2708/2708 [==============================] - 0s 10us/step - loss: 1.9440 - weighted_acc: 0.3071 - val_loss: 1.9443 - val_weighted_acc: 0.3100
Epoch 5/100
2708/2708 [==============================] - 0s 10us/step - loss: 1.9430 - weighted_acc: 0.3071 - val_loss: 1.9440 - val_weighted_acc: 0.3500
Epoch 6/100
2708/2708 [==============================] - 0s 9us/step - loss: 1.9421 - weighted_acc: 0.3929 - val_loss: 1.9436 - val_we

Epoch 52/100
2708/2708 [==============================] - 0s 11us/step - loss: 1.8752 - weighted_acc: 0.8071 - val_loss: 1.9083 - val_weighted_acc: 0.7060
Epoch 53/100
2708/2708 [==============================] - 0s 9us/step - loss: 1.8777 - weighted_acc: 0.7500 - val_loss: 1.9073 - val_weighted_acc: 0.7060
Epoch 54/100
2708/2708 [==============================] - 0s 10us/step - loss: 1.8720 - weighted_acc: 0.7929 - val_loss: 1.9063 - val_weighted_acc: 0.7040
Epoch 55/100
2708/2708 [==============================] - 0s 9us/step - loss: 1.8699 - weighted_acc: 0.8500 - val_loss: 1.9053 - val_weighted_acc: 0.7060
Epoch 56/100
2708/2708 [==============================] - 0s 9us/step - loss: 1.8695 - weighted_acc: 0.7857 - val_loss: 1.9044 - val_weighted_acc: 0.7040
Epoch 57/100
2708/2708 [==============================] - 0s 9us/step - loss: 1.8692 - weighted_acc: 0.8143 - val_loss: 1.9034 - val_weighted_acc: 0.7020
Epoch 58/100
2708/2708 [==============================] - 0s 9us/step - lo

In [9]:
# Evaluate model
eval_results = model.evaluate([X, A],
                              y_test,
                              sample_weight=test_mask,
                              batch_size=N)
print('Done.\n'
      'Test loss: {}\n'
      'Test accuracy: {}'.format(*eval_results))


2708/2708 [==============================] - 0s 6us/step
Done.
Test loss: 1.8529187440872192
Test accuracy: 0.7399999499320984


### Data loader:

https://github.com/danielegrattarola/spektral/blob/master/spektral/datasets/qm9.py

https://github.com/danielegrattarola/spektral/blob/master/examples/regression_molecules.py

In [11]:
import matplotlib.pyplot as plt
import numpy as np
from keras.callbacks import EarlyStopping
from keras.layers import Input, Dense
from keras.models import Model
from keras.optimizers import Adam
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

from spektral.datasets import qm9
from spektral.layers import EdgeConditionedConv, GlobalAttentionPool
from spektral.utils import label_to_one_hot

In [16]:
adj, nf, ef, y = qm9.load_data(return_type='numpy',
                               nf_keys='atomic_num',
                               ef_keys='type',
                               self_loops=True)

  0%|          | 0/133885 [00:00<?, ?it/s]

Loading QM9 dataset.
Reading SDF


100%|██████████| 133885/133885 [00:43<00:00, 3060.38it/s]


In [20]:
y_array = y[['cv']].values  # Heat capacity at 298.15K
y_array

array([[ 6.469],
       [ 6.316],
       [ 6.002],
       ...,
       [23.972],
       [24.796],
       [23.434]])

In [21]:
# Preprocessing
uniq_nf = np.unique(nf)
nf = label_to_one_hot(nf, uniq_nf)
uniq_ef = np.unique(ef)
ef = label_to_one_hot(ef, uniq_ef)
y_array_scaled = StandardScaler().fit_transform(y_array).reshape(-1, y_array.shape[-1])

In [34]:
# Train/test split
adj_train, adj_test, \
nf_train, nf_test,   \
ef_train, ef_test,   \
y_train, y_test = train_test_split(adj, nf, ef, y_array_scaled, test_size=0.1)

In [42]:
# Parameters
N = nf.shape[-2]          # Number of nodes in the graphs
F = nf.shape[-1]          # Node features dimensionality
S = ef.shape[-1]          # Edge features dimensionality
n_out = y_array.shape[-1]       # Dimensionality of the target
learning_rate = 1e-3      # Learning rate for SGD
epochs = 25               # Number of training epochs
batch_size = 64           # Batch size
es_patience = 5           # Patience fot early stopping

In [43]:
# Model definition
nf_in = Input(shape=(N, F))
adj_in = Input(shape=(N, N))
ef_in = Input(shape=(N, N, S))

gc1 = EdgeConditionedConv(32, activation='relu')([nf_in, adj_in, ef_in])
gc2 = EdgeConditionedConv(64, activation='relu')([gc1, adj_in, ef_in])
pool = GlobalAttentionPool(128)(gc2)
dense1 = Dense(128, activation='relu')(pool)

output = Dense(n_out)(dense1)

In [44]:
# Build model
model = Model(inputs=[nf_in, adj_in, ef_in], outputs=output)
optimizer = Adam(lr=learning_rate)
model.compile(optimizer=optimizer, loss='mse')
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_14 (InputLayer)           (None, 29, 6)        0                                            
__________________________________________________________________________________________________
input_15 (InputLayer)           (None, 29, 29)       0                                            
__________________________________________________________________________________________________
input_16 (InputLayer)           (None, 29, 29, 4)    0                                            
__________________________________________________________________________________________________
edge_conditioned_conv_5 (EdgeCo (None, 29, 32)       992         input_14[0][0]                   
                                                                 input_15[0][0]                   
          

In [45]:
# Callbacks
es_callback = EarlyStopping(monitor='val_loss', patience=es_patience)

In [46]:
nf_train.shape, adj_train.shape, ef_train.shape, y_train.shape

((120496, 29, 6), (120496, 29, 29), (120496, 29, 29, 4), (120496, 1))

In [ ]:
# Train model
model.fit([nf_train, adj_train, ef_train],
          y_train,
          batch_size=batch_size,
          validation_split=0.1,
          epochs=epochs,
          callbacks=[es_callback])

Train on 108446 samples, validate on 12050 samples
Epoch 1/25
108446/108446 [==============================] - 409s 4ms/step - loss: 0.0619 - val_loss: 0.0305
Epoch 2/25
 33088/108446 [========>.....................] - ETA: 4:33 - loss: 0.0344